In [1]:
# from datasets import load_dataset
# raw_dataset = load_dataset('super_glue', 'cb', cache_dir="../datasets/.cache/huggingface_datasets")
from datasets import load_from_disk, Dataset,concatenate_datasets
import pandas as pd
df=pd.read_csv("data/Train.csv")
dataset = Dataset.from_pandas(df)

/home/sdm/anaconda3/envs/openprompt/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
raw_dataset=dataset.train_test_split(test_size=0.2)
raw_dataset2=raw_dataset['test'].train_test_split(test_size=0.5)
raw_dataset,raw_dataset2

(DatasetDict({
     train: Dataset({
         features: ['Data', 'Label'],
         num_rows: 10060
     })
     test: Dataset({
         features: ['Data', 'Label'],
         num_rows: 2515
     })
 }),
 DatasetDict({
     train: Dataset({
         features: ['Data', 'Label'],
         num_rows: 1257
     })
     test: Dataset({
         features: ['Data', 'Label'],
         num_rows: 1258
     })
 }))

In [3]:
from datasets.dataset_dict import DatasetDict
from datasets import Dataset
d = {'train':Dataset.from_dict({'Data':raw_dataset['train']['Data'],'Label':raw_dataset['train']['Label']}),'validation':Dataset.from_dict({'Data':raw_dataset2['train']['Data'],'Label':raw_dataset2['train']['Label']}),'test':Dataset.from_dict({'Data':raw_dataset2['train']['Data'],'Label':raw_dataset2['train']['Label']})}
raw_datasets=DatasetDict(d)
print(raw_datasets)

DatasetDict({
    train: Dataset({
        features: ['Data', 'Label'],
        num_rows: 10060
    })
    validation: Dataset({
        features: ['Data', 'Label'],
        num_rows: 1257
    })
    test: Dataset({
        features: ['Data', 'Label'],
        num_rows: 1257
    })
})


In [4]:
from openprompt.data_utils import InputExample

dataset = {}
for split in ['train', 'validation', 'test']:
    dataset[split] = []
    for idx,data in enumerate(raw_datasets[split]):
        input_example = InputExample(text_a = data['Data'], label=int(data['Label']), guid=idx)
        dataset[split].append(input_example)
print(dataset['train'][0])

{
  "guid": 0,
  "label": 0,
  "meta": {},
  "text_a": "\u09a6\u09be\u09a6\u09be \u098f\u09ac\u09be\u09b0 \u098f\u0995\u09ac\u09be\u09b0 \u09ac\u09be\u09b2\u09cd\u09b2\u09be\u09b0\u09be\u09ae \u09ae\u09c1\u09b2\u09cd\u09b2\u09bf\u0995 \u098f\u09a8\u09cd\u09a1 \u09b0\u09be\u09a7\u09be\u09b0\u09ae\u09a3 \u09ae\u09c1\u09b2\u09cd\u09b2\u09bf\u0995 \u098f\u09b0 \u09a6\u09cb\u0995\u09be\u09a8 \u098f\u09b0 \u09b0\u09b8\u0997\u09cb\u09b2\u09cd\u09b2\u09be \u09a8\u09be \u0997\u09c7\u09b2\u09c7 \u09ac\u09bf\u09b0\u09be\u099f \u098f\u0995\u099f\u09be \u099c\u09bf\u09a8\u09bf\u09b8 \u0995\u09bf\u09a8\u09cd\u09a4\u09c1 \u09ae\u09bf\u09b8 \u0995\u09b0\u09c7\u09af\u09be\u09ac\u09c7\u09a8",
  "text_b": "",
  "tgt_text": null
}



In [5]:
from openprompt.plms import load_plm
plm, tokenizer, model_config, WrapperClass = load_plm("t5", "google/mt5-base")

/home/sdm/anaconda3/envs/openprompt/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
You are using a model of type mt5 to instantiate a model of type t5. This is not supported for all configurations of models and can yield errors.
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [6]:
from openprompt.prompts import ManualTemplate
# template_text = '{"placeholder":"text_a"}. এটা হল {"mask"}.'
template_text = '{"placeholder":"text_a"}. sentiment: {"mask"}.'
# template_text = '[CLS] {"placeholder":"text_a"}. Sentiment: {"mask"}. [SEP]'
mytemplate = ManualTemplate(tokenizer=tokenizer, text=template_text)

In [7]:
wrapped_example = mytemplate.wrap_one_example(dataset['train'][0])
print(wrapped_example)

[[{'text': 'দাদা এবার একবার বাল্লারাম মুল্লিক এন্ড রাধারমণ মুল্লিক এর দোকান এর রসগোল্লা না গেলে বিরাট একটা জিনিস কিন্তু মিস করেযাবেন', 'loss_ids': 0, 'shortenable_ids': 1}, {'text': '. sentiment:', 'loss_ids': 0, 'shortenable_ids': 0}, {'text': '<mask>', 'loss_ids': 1, 'shortenable_ids': 0}, {'text': '.', 'loss_ids': 0, 'shortenable_ids': 0}], {'guid': 0, 'label': 0}]


In [8]:
tokenizer.add_special_tokens({'additional_special_tokens': ['<mask>']})
print("Additional special tokens:", tokenizer.additional_special_tokens) 
print("Additional special tokens ids:", tokenizer.additional_special_tokens_ids)

Additional special tokens: ['<mask>']
Additional special tokens ids: [250100]


In [9]:
# wrapped_t5tokenizer = WrapperClass(max_seq_length=128, decoder_max_length=3, tokenizer=tokenizer,truncate_method="head")
# or
from openprompt.plms import T5TokenizerWrapper
wrapped_t5tokenizer= T5TokenizerWrapper(max_seq_length=128, decoder_max_length=3, tokenizer=tokenizer,truncate_method="head")
# tokenized_example = wrapped_t5tokenizer.tokenize_one_example(wrapped_example, teacher_forcing=False)

In [10]:
from openprompt import PromptDataLoader

train_dataloader = PromptDataLoader(dataset=dataset["train"], template=mytemplate, tokenizer=tokenizer,
    tokenizer_wrapper_class=WrapperClass, max_seq_length=256, decoder_max_length=3,
    batch_size=4,shuffle=True, teacher_forcing=False, predict_eos_token=False,
    truncate_method="head")

tokenizing: 10060it [00:01, 6265.86it/s]


In [11]:
# tokenized_example = wrapped_t5tokenizer.tokenize_one_example(wrapped_example, teacher_forcing=False)
# print(tokenized_example)
# print(tokenizer.convert_ids_to_tokens(tokenized_example['input_ids']))
# print(tokenizer.convert_ids_to_tokens(tokenized_example['decoder_input_ids']))

In [12]:
from openprompt.prompts import ManualVerbalizer
import torch

# for example the verbalizer contains multiple label words in each class
myverbalizer = ManualVerbalizer(tokenizer, num_classes=3,
                        label_words=[["Neutral"], ["Positive"], ["Negative"]])

print(myverbalizer.label_words_ids)
logits = torch.randn(2,len(tokenizer)) # creating a pseudo output from the plm, and
print(myverbalizer.process_logits(logits)) # see what the verbalizer do

Parameter containing:
tensor([[[156989,      0]],

        [[   259, 137088]],

        [[153418,    265]]])
tensor([[-1.4308, -0.8622, -1.0828],
        [-2.1807, -0.2454, -2.2571]])


In [13]:
from openprompt import PromptForClassification

use_cuda = True
prompt_model = PromptForClassification(plm=plm,template=mytemplate, verbalizer=myverbalizer, freeze_plm=False)
if use_cuda:
    prompt_model=  prompt_model.cuda()

# Now the training is standard
from transformers import  AdamW, get_linear_schedule_with_warmup
loss_func = torch.nn.CrossEntropyLoss()
no_decay = ['bias', 'LayerNorm.weight']
# it's always good practice to set no decay to biase and LayerNorm parameters
optimizer_grouped_parameters = [
    {'params': [p for n, p in prompt_model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in prompt_model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=1e-4)

/home/sdm/anaconda3/envs/openprompt/lib/python3.8/site-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [14]:
for epoch in range(10):
    tot_loss = 0
    for step, inputs in enumerate(train_dataloader):
        if use_cuda:
            inputs = inputs.cuda()
        logits = prompt_model(inputs)
        labels = inputs['label']
        loss = loss_func(logits, labels)
        loss.backward()
        tot_loss += loss.item()
        optimizer.step()
        optimizer.zero_grad()
        if step %100 ==1:
            print("Epoch {}, average loss: {}".format(epoch, tot_loss/(step+1)), flush=True)

Epoch 0, average loss: 11.264082431793213
Epoch 0, average loss: 1.7002377524679781
Epoch 0, average loss: 1.3424039277994986
Epoch 0, average loss: 1.201624226343158
Epoch 0, average loss: 1.1289077883780891
Epoch 0, average loss: 1.080813111626057
Epoch 0, average loss: 1.0393867160369985
Epoch 0, average loss: 1.0199579295630639
Epoch 0, average loss: 0.9956281465429468
Epoch 0, average loss: 0.9776344262476506
Epoch 0, average loss: 0.9663131166390793
Epoch 0, average loss: 0.952214144941034
Epoch 0, average loss: 0.9401341166700579
Epoch 0, average loss: 0.9344097701131656
Epoch 0, average loss: 0.9269766013075553
Epoch 0, average loss: 0.9203895375866706
Epoch 0, average loss: 0.910217711415965
Epoch 0, average loss: 0.8997490011947941
Epoch 0, average loss: 0.8914469151957153
Epoch 0, average loss: 0.8828864262882937
Epoch 0, average loss: 0.8752928335465245
Epoch 0, average loss: 0.8705877384505847
Epoch 0, average loss: 0.8648724835230669
Epoch 0, average loss: 0.8612174421700

In [15]:
validation_dataloader = PromptDataLoader(dataset=dataset["validation"], template=mytemplate, tokenizer=tokenizer,
    tokenizer_wrapper_class=WrapperClass, max_seq_length=256, decoder_max_length=3,
    batch_size=4,shuffle=False, teacher_forcing=False, predict_eos_token=False,
    truncate_method="head")

tokenizing: 1257it [00:00, 6506.77it/s]


In [16]:
allpreds = []
alllabels = []
for step, inputs in enumerate(validation_dataloader):
    if use_cuda:
        inputs = inputs.cuda()
    logits = prompt_model(inputs)
    labels = inputs['label']
    alllabels.extend(labels.cpu().tolist())
    allpreds.extend(torch.argmax(logits, dim=-1).cpu().tolist())

acc = sum([int(i==j) for i,j in zip(allpreds, alllabels)])/len(allpreds)
print(acc)

0.7024661893396977


In [ ]:
from typing import List, Optional
from transformers.modeling_utils import PreTrainedModel
from .utils import TokenizerWrapper
from transformers.tokenization_utils import PreTrainedTokenizer
from .mlm import MLMTokenizerWrapper
from .seq2seq import T5LMTokenizerWrapper, T5TokenizerWrapper
from .lm import LMTokenizerWrapper
from transformers import BertConfig, BertTokenizer, BertModel, BertForMaskedLM, \
                         RobertaConfig, RobertaTokenizer, RobertaModel, RobertaForMaskedLM, \
                         AlbertTokenizer, AlbertConfig, AlbertModel, AlbertForMaskedLM, \
                         T5Config, T5Tokenizer, T5ForConditionalGeneration, \
                         OpenAIGPTTokenizer, OpenAIGPTLMHeadModel, OpenAIGPTConfig, \
                         GPT2Config, GPT2Tokenizer, GPT2LMHeadModel, \
                         OPTConfig, OPTForCausalLM
from collections import namedtuple
from yacs.config import CfgNode

from openprompt.utils.logging import logger


ModelClass = namedtuple("ModelClass", ('config', 'tokenizer', 'model','wrapper'))

_MODEL_CLASSES = {
    'bert': ModelClass(**{
        'config': BertConfig,
        'tokenizer': BertTokenizer,
        'model':BertForMaskedLM,
        'wrapper': MLMTokenizerWrapper,
    }),
    'roberta': ModelClass(**{
        'config': RobertaConfig,
        'tokenizer': RobertaTokenizer,
        'model':RobertaForMaskedLM,
        'wrapper': MLMTokenizerWrapper
    }),
    'albert': ModelClass(**{
        'config': AlbertConfig,
        'tokenizer': AlbertTokenizer,
        'model': AlbertForMaskedLM,
        'wrapper': MLMTokenizerWrapper
    }),
    'gpt': ModelClass(**{
        'config': OpenAIGPTConfig,
        'tokenizer': OpenAIGPTTokenizer,
        'model': OpenAIGPTLMHeadModel,
        'wrapper': LMTokenizerWrapper
    }),
    'gpt2': ModelClass(**{
        'config': GPT2Config,
        'tokenizer': GPT2Tokenizer,
        'model': GPT2LMHeadModel,
        'wrapper': LMTokenizerWrapper
    }),
    't5':ModelClass(**{
        'config': T5Config,
        'tokenizer': T5Tokenizer,
        'model': T5ForConditionalGeneration,
        'wrapper': T5TokenizerWrapper
    }),
    't5-lm':ModelClass(**{
        'config': T5Config,
        'tokenizer': T5Tokenizer,
        'model': T5ForConditionalGeneration,
        'wrapper': T5LMTokenizerWrapper,
    }),
    'opt': ModelClass(**{
        'config': OPTConfig,
        'tokenizer': GPT2Tokenizer,
        'model': OPTForCausalLM,
        'wrapper': LMTokenizerWrapper,
    }),
}


def get_model_class(plm_type: str):
    return _MODEL_CLASSES[plm_type]


def load_plm(model_name, model_path, specials_to_add = None):
    r"""A plm loader using a global config.
    It will load the model, tokenizer, and config simulatenously.

    Args:
        config (:obj:`CfgNode`): The global config from the CfgNode.

    Returns:
        :obj:`PreTrainedModel`: The pretrained model.
        :obj:`tokenizer`: The pretrained tokenizer.
        :obj:`model_config`: The config of the pretrained model.
        :obj:`wrapper`: The wrapper class of this plm.
    """
    model_class = get_model_class(plm_type = model_name)
    model_config = model_class.config.from_pretrained(model_path)
    # you can change huggingface model_config here
    # if 't5'  in model_name: # remove dropout according to PPT~\ref{}
    #     model_config.dropout_rate = 0.0
    if 'gpt' in model_name: # add pad token for gpt
        specials_to_add = ["<pad>"]
        # model_config.attn_pdrop = 0.0
        # model_config.resid_pdrop = 0.0
        # model_config.embd_pdrop = 0.0
    model = model_class.model.from_pretrained(model_path, config=model_config)
    if model_name=="roberta":
        from transformers import AutoTokenizer, AutoModelForMaskedLM
        tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')
    else:    
        tokenizer = model_class.tokenizer.from_pretrained(model_path)
    wrapper = model_class.wrapper


    model, tokenizer = add_special_tokens(model, tokenizer, specials_to_add=specials_to_add)

    if 'opt' in model_name:
        tokenizer.add_bos_token=False
    return model, tokenizer, model_config, wrapper




def load_plm_from_config(config: CfgNode):
    r"""A plm loader using a global config.
    It will load the model, tokenizer, and config simulatenously.

    Args:
        config (:obj:`CfgNode`): The global config from the CfgNode.

    Returns:
        :obj:`PreTrainedModel`: The pretrained model.
        :obj:`tokenizer`: The pretrained tokenizer.
        :obj:`model_config`: The config of the pretrained model.
        :obj:`model_config`: The wrapper class of this plm.
    """
    plm_config = config.plm
    model_class = get_model_class(plm_type = plm_config.model_name)
    model_config = model_class.config.from_pretrained(plm_config.model_path)
    # you can change huggingface model_config here
    # if 't5'  in plm_config.model_name: # remove dropout according to PPT~\ref{}
    #     model_config.dropout_rate = 0.0
    if 'gpt' in plm_config.model_name: # add pad token for gpt
        if "<pad>" not in config.plm.specials_to_add:
            config.plm.specials_to_add.append("<pad>")
    model = model_class.model.from_pretrained(plm_config.model_path, config=model_config)
    tokenizer = model_class.tokenizer.from_pretrained(plm_config.model_path)
    wrapper = model_class.wrapper
    model, tokenizer = add_special_tokens(model, tokenizer, specials_to_add=config.plm.specials_to_add)
    return model, tokenizer, model_config, wrapper

def add_special_tokens(model: PreTrainedModel,
                       tokenizer: PreTrainedTokenizer,
                       specials_to_add: Optional[List[str]] = None):
    r"""add the special_tokens to tokenizer if the special token
    is not in the tokenizer.

    Args:
        model (:obj:`PreTrainedModel`): The pretrained model to resize embedding
                after adding special tokens.
        tokenizer (:obj:`PreTrainedTokenizer`): The pretrained tokenizer to add special tokens.
        specials_to_add: (:obj:`List[str]`, optional): The special tokens to be added. Defaults to pad token.

    Returns:
        The resized model, The tokenizer with the added special tokens.

    """
    if specials_to_add is None:
        return model, tokenizer
    for token in specials_to_add:
        if "pad" in token.lower():
            if tokenizer.pad_token is None:
                tokenizer.add_special_tokens({'pad_token': token})
                model.resize_token_embeddings(len(tokenizer))
                logger.info("pad token is None, set to id {}".format(tokenizer.pad_token_id))
    return model, tokenizer



